# Getting started with DoWhy: A simple example
This is quick introduction to DoWhy causal inference library.
We will load in a sample dataset and estimate causal effect from a (pre-specified)treatment variable to a (pre-specified) outcome variable.

First, let us add required path for python to find DoWhy code and load required packages.

In [1]:
import os, sys
sys.path.append(os.path.abspath("../../"))

In [2]:
import numpy as np
import pandas as pd

import dowhy
from dowhy.do_why import CausalModel
import dowhy.datasets 

Let us first load a dataset. For simplicity, we simulate a dataset with linear relationships between common causes and treatment, and common causes and outcome. 

Beta is the true causal effect. 

In [3]:
data = dowhy.datasets.linear_dataset(beta=10,
        num_common_causes=5,
        num_instruments = 2,
        num_samples=10000, 
        treatment_is_binary=True)
df = data["df"]
print(df.head())
print(data["dot_graph"])
print("\n")
print(data["gml_graph"])

    Z0        Z1        X0        X1        X2        X3        X4    v  \
0  0.0  0.236335  0.266520 -1.282418  0.602125  0.631051 -0.117089  1.0   
1  0.0  0.644808 -0.098507  1.124507  1.039807 -1.105611 -0.293460  1.0   
2  0.0  0.283139 -1.316949 -1.048244  1.198808 -0.362258  2.357232  1.0   
3  0.0  0.818932 -2.352635 -0.024943 -0.126628 -0.903706 -0.055516  0.0   
4  0.0  0.485443  0.117577 -0.683765  0.250597  0.002824  0.366975  0.0   

           y  
0   9.427494  
1  13.830990  
2   8.941143  
3 -11.145470  
4  -0.987937  
digraph { v ->y; U[label="Unobserved Confounders"]; U->v; U->y;X0-> v; X1-> v; X2-> v; X3-> v; X4-> v;X0-> y; X1-> y; X2-> y; X3-> y; X4-> y;Z0-> v; Z1-> v;}


graph[node[ id "v" label "v"]node[ id "y" label "y"]node[ id "Unobserved Confounders" label "Unobserved Confounders"]edge[source "v" target "y"]edge[source "Unobserved Confounders" target "v"]edge[source "Unobserved Confounders" target "y"]node[ id "X0" label "X0"] edge[ source "X0" target "v"] nod

Note that we are using a pandas dataframe to load the data. At present, DoWhy only supports pandas dataframe as input.

## Interface 1 (recommended): Input causal graph

We now input a causal graph in the DOT graph format.

In [6]:
# With graph
model=CausalModel(
        data = df,
        treatment=data["treatment_name"],
        outcome=data["outcome_name"],
        graph=data["dot_graph"]
        )

Error: Pygraphviz cannot be loaded. No module named 'pygraphviz'
Trying pydot ...
Error: Pydot cannot be loaded. 'list' object has no attribute 'get_strict'


AttributeError: 'list' object has no attribute 'get_strict'

In [6]:
model.view_model()

Using Matplotlib for plotting


<img src="causal_model.png">

We get the same causal graph. Now identification and estimation is done as before.

In [ ]:
identified_estimand = model.identify_effect()
print(identified_estimand)

In [ ]:
causal_estimate = model.estimate_effect(identified_estimand,
        method_name="backdoor.linear_regression")
print(causal_estimate)
print("Causal Estimate is " + str(causal_estimate.value))

## Interface 2: Specify common causes and instruments

In [ ]:
# Without graph                                       
model= CausalModel(                             
        data=df,                                      
        treatment=data["treatment_name"],             
        outcome=data["outcome_name"],                 
        common_causes=data["common_causes_names"])                         

In [ ]:
model.view_model()

<img src="causal_model.png" />

The above causal graph shows the assumptions encoded in the cauasl model. We can now use this graph to first identify 
the causal effect (go from a causal estimand to a probability expression), and then estimate the causal effect.

**DoWhy philosophy: Keep identification and estimation separate**

Identification can be achieved without access to data, only the graph. This results in an expression to computed. This expression can then be computed using the available data in the estimation step.
Important to understand that these are orthogonal steps.

* Identification

In [ ]:
identified_estimand = model.identify_effect()                         

* Estimation

In [ ]:
estimate = model.estimate_effect(identified_estimand,
        method_name="backdoor.linear_regression", 
        test_significance=True
        )         
print(estimate)
print("Causal Estimate is " + str(estimate.value))

## Refuting the estimate

Now refuting the obtained estimate.

### Adding a random common cause variable

In [ ]:
res_random=model.refute_estimate(identified_estimand, estimate, method_name="random_common_cause")
print(res_random)

### Replacing treatment with a random (placebo) variable

In [ ]:
res_placebo=model.refute_estimate(identified_estimand, estimate,
        method_name="placebo_treatment_refuter", placebo_type="permute")
print(res_placebo)

### Removing a random subset of the data

In [ ]:
res_subset=model.refute_estimate(identified_estimand, estimate,
        method_name="data_subset_refuter", subset_fraction=0.9)
print(res_subset)


As you can see, the linear regression estimator is very sensitive to simple refutations.